In [1]:
### Importing necessary libraries
import os
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import time

### Loading matlab files in Python using Scipy
import scipy.io
mat = scipy.io.loadmat('Brain.mat')
T1, label = mat['T1'], mat['label']

### T1 is the array with the input MRI images, 10 samples each of size 362x434 pixels.
T1.shape
images = []
labels = []
for i in range(10):
    img = T1[:,:,i]
    ### standardising the float values between int of range(0-255)
    img = ((img - img.min()) * (1/(img.max() - img.min()) * 255)).astype('uint8')
    images.append(img)
    lab = label[:,:,i]
    labels.append(lab)

In [3]:
input_image = images[5]
label_image = labels[5]

In [ ]:
# TASK 2: Evaluating the performance
# Before we move on to different approaches in obtaining the segmentation, 
#we will first define the evaluation metrics and get an accuracy measure with the groundtruth and verify the logics. 

In [6]:
def pixelwise_IOU_label(input_image, ground_truth, label_class = 1):
    mask1 = input_image == label_class
    mask2 = ground_truth == label_class
    iou_score = IOU_binary(mask1, mask2)
    return iou_score

In [7]:
def IOU_binary(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [8]:
def dice_binary(mask1, mask2):
    mask1_pos = mask1.astype(np.float32)
    mask2_pos = mask2.astype(np.float32)
    dice = 2 * np.sum(mask1_pos * mask2_pos) / (np.sum(mask1_pos) + np.sum(mask2_pos))
    return dice

In [9]:
def score_image(predict, label):
    label_index = {0:'air',1:'skin',2:'skull',3:'csf',4:'gray matter',5:'white matter'}
    scores = []
    for pix_val, category in label_index.items():
        predict_mask = (predict == pix_val)
        label_mask = (label == pix_val)        
        dice = dice_binary(predict_mask, label_mask)
        scores.append(dice)
    scores.append(sum(scores)/len(scores))
    df = pd.DataFrame(columns=['air','skin','skull','csf','gray matter','white matter','0_mean'])
    df.loc[0] = scores
    return df

In [18]:
import pandas as pd

def score_images(predicts, labels):
    df = pd.DataFrame(columns=['air','skin','skull','csf','gray matter','white matter','0_mean'])
    for i, (predict, label) in enumerate(zip(predicts,labels)):
        scores = score_image(predict,label)
#         scores.append(sum(scores)/len(scores))
        df = pd.concat([df, scores])
#     print("Dice scores:")
#     for label in df.columns:
#         print(f"{label} : {df[label].mean()}")
    return df
df = score_images(labels,labels)
# df

In [ ]:
# TASK 1: MRI Segmentation

In [ ]:
## MRI Segementation - Approach 1 : K-means Clustering

In [19]:
def kmeans_segmentation(input_image, no_of_classes = 6):
    
    h, w = input_image.shape
    # reshape to 1D array
    image_2d = input_image.reshape(h*w,1)

    image_2d = np.float32(image_2d)
    ### using cv2 kmeans
    ### In criteria, we first set the algorithm termination criteria, either max no of iterations or desired result 
    ### here max iterations is set to 50, and the max score is 1.0
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 1.0)
    K = no_of_classes
    attempts = 100
    ### calling kmeans
    ret, label, center = cv2.kmeans(image_2d,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((input_image.shape))
    
    ### assigning each cluster to a label based on the range of pixels based on observation
    out_image = np.zeros_like(result_image)
    out_image = np.where(((result_image>45) & (result_image<55)), 1, out_image )
    out_image = np.where(((result_image>75) & (result_image<95)), 3, out_image )
    out_image = np.where(((result_image>115) & (result_image<135)), 4, out_image )
    out_image = np.where(((result_image>145) & (result_image<160)), 5, out_image )
    out_image = np.where(((result_image>200) & (result_image<210)), 2, out_image )
    
    return out_image

out_image = kmeans_segmentation(input_image)
score_image(out_image,label_image)

,air,skin,skull,csf,gray matter,white matter,0_mean
0,0.915257,0.249211,0.0,0.299259,0.820272,0.950255,0.539042


In [ ]:
## MRI Segmentation - Approach 2 : Class by class Method: Filtering, Thresholding & Morphological operations

In [20]:
### function to filter labels with less than 10% area
def filter_noise_label(out_label):
    for i in np.unique(out_label):
        if np.count_nonzero(out_label==i)<(0.1*out_label.shape[0]*out_label.shape[1]):
            out_label = np.where(out_label==i,0,out_label)
    return out_label
### class wise segmentation
def class_wise_segmentation(input_image, intermediate = False):
    ### applying a 5x5 Gaussian blur to smoothen the image
    blur_image = cv2.GaussianBlur(input_image.copy(),(5,5),cv2.BORDER_DEFAULT)
    ### creating an empty image with the same shape as input_image
    out_image = np.zeros_like(input_image)
    ### air
    ### thresholding with value of 17 for getting air
    _, air = cv2.threshold(blur_image, 17 , 255, cv2.THRESH_BINARY)
    ### applying a layer of closing and dilation
    kernel = np.ones((5,5),dtype=np.uint8)
    air_mask = ~cv2.morphologyEx(air, cv2.MORPH_CLOSE, kernel,iterations = 1)
    air_mask = cv2.morphologyEx(air_mask, cv2.MORPH_DILATE, kernel, iterations =5)

    ### skin
    ### thresholding with a value of 50
    th, skin = cv2.threshold(blur_image, 50, 255, cv2.THRESH_BINARY)
    ### applying opening
    kernel = np.ones((5,5),dtype=np.uint8)
    dst2_open = cv2.morphologyEx(skin, cv2.MORPH_OPEN, kernel,iterations = 1)
    
    ### calling connected components function
    retval, out_label, stats, centroids = cv2.connectedComponentsWithStats(dst2_open)
    ### taking only skin
    skin_mask = (out_label==1).astype(np.uint8)
    ### applying dilation
    kernel = np.ones((3,3),dtype=np.uint8)
    skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_DILATE, kernel,iterations = 1)
    ### filtering noise labels from the connected components output
    out_label = filter_noise_label(out_label)
    ### masking the non_air parts by 6 to avoid being considered as air
    out_image = np.where(air_mask!=0,0,6)
    ### skin added with value 1 in the out image
    out_image = np.where(skin_mask!=0,1,out_image)

    ## skull
    ### region inside skull is region which is disjoint with the union of the skin and the air mask
    inside_skin = np.zeros_like(out_image)
    inside_skin = np.where(out_image < 5,255,0)
    ### applying dilation
    kernel = np.ones((9,9),dtype=np.uint8)
    inside_skin = cv2.morphologyEx(inside_skin.astype(np.uint8), cv2.MORPH_CLOSE, kernel,iterations = 1)
    ### accessing the brain component with label 2
    brain = np.where(out_label==2,255,0)
    ### applying dilation
    kernel = np.ones((7,7),dtype=np.uint8)
    brain_filled = cv2.morphologyEx(brain.astype(np.uint8), cv2.MORPH_CLOSE, kernel,iterations = 5)
    ### skull is the region between skin and brain
    skull_mask = np.bitwise_xor(~inside_skin, brain_filled)
    ### marking the pixels as skull with 2 in the out_image
    out_image = np.where(skull_mask!=0,2,out_image)
    
    ### csf
    ### thresholding with 100
    _, csf = cv2.threshold(input_image, 100 , 255, cv2.THRESH_BINARY)
    ### finding region inside skull which is disjoint to the union of the skin,air,and skull masks
    inside_skull = np.zeros_like(out_image)
    inside_skull = np.where(out_image<5, 255, 0).astype(np.uint8)
    ### applying closing
    kernel = np.ones((7,7),dtype=np.uint8)
    inside_skull = cv2.morphologyEx(inside_skull, cv2.MORPH_CLOSE, kernel,iterations = 1)
    ### filtering regions inside skull and the thresholded csf image
    csf = np.bitwise_xor(csf,~inside_skull)
    csf = np.where(out_image>5, csf, 0)
    
    ### finding the bounding box of the detected brain
    retval, out_label_csf, stats, centroids = cv2.connectedComponentsWithStats(csf)
    x, y, w, h, area = sorted(stats,key = lambda x:x[4],reverse = True)[1]
    
    ### from the images, we can see that the csf mask obtained need to retain its pixels in the borders
    ### and need to erode the parts inside which is identified as csf
    ### we will use a selective erosion by masking
    
    # Supervised Eliptical Erosion 
    
    ellipse_mask = np.zeros_like(csf)
    ### finding a large ellipse inside the brain to erode
    cv2.ellipse(ellipse_mask, ((int((x+w+x)/2),int((y+y+h)/2)), (0.95*w,0.95*h), 0.0), (255, 255, 255), -1);
    ### making a mask of the ellipse and copying the contents of csf inside the mask
    csf_centre_thin = np.where(ellipse_mask & csf!=0, csf, 0)
    ### taking the rest of the region outside the ellipse
    csf_rest = np.where(ellipse_mask & csf==0, csf,0)
    ### eroding the content inside the ellipse
    kernel = np.ones((3,3),dtype=np.uint8)
    csf_centre_thin = cv2.morphologyEx(csf_centre_thin, cv2.MORPH_ERODE, kernel,iterations = 1)
    ### merging the newly eroded content inside the ellipse and the original content outside ellipse
    csf = csf_rest + csf_centre_thin 
    
    out_image = np.where(csf!=0, 3, out_image)
    
    ### gray & white matter
    ### gray matter is directly obtained from thresholding at 140 and removing the non_brain regions
    _, gray_matter = cv2.threshold(blur_image, 140 , 255, cv2.THRESH_BINARY)
    gray_matter = np.where(~inside_skull>0, gray_matter,0)
    out_image = np.where(gray_matter!=0, 5, out_image)
    ### rest of the regions which does not belong to any category is the white matter
    out_image = np.where(out_image>5, 4, out_image)
    return [out_image,inside_skull, air_mask, inside_skin, skin_mask, skull_mask]

In [36]:
### Approach 3: Combining Kmeans and Class_by_Class_segmentation results
def combined_kmeans_and_class_by_class(input_image, masks, kmeans_out_image):
    ### accesing the intermediate outputs from class_by_class segmentation
    out_image,inside_skull, air_mask, inside_skin, skin_mask, skull_mask = masks
    ### creating new output image, masks for new gray and white matter
    new_image = np.zeros_like(out_image)
    combined_gray_matter = np.zeros_like(out_image)
    combined_white_matter = np.zeros_like(out_image)

    ### taking gray matter from kmeans after filtering the non_brain regions
    gray_matter_kmeans = (kmeans_out_image==4)
    combined_gray_matter = np.where(~inside_skull!=0, gray_matter_kmeans, combined_gray_matter)
    ### taking white matter from kmeans after filtering the non_brain regions
    white_matter_kmeans = (kmeans_out_image==5)
    combined_white_matter = np.where(~inside_skull!=0, white_matter_kmeans, new_image)
    ### labeling gray and white matter
    new_image = np.where(combined_gray_matter!=0, 4, new_image)
    new_image = np.where(combined_white_matter!=0, 5, new_image)

    ### improving skull mask from kmeans
    air_kmeans = (kmeans_out_image==0)
    ### taking the thresholded air_mask
    new_skull_mask= np.bitwise_or(air_kmeans,air_mask)
    new_skull_mask = np.bitwise_and(new_skull_mask,~inside_skin)
    ### applying dilation
    kernel = np.ones((3,3),dtype=np.uint8)
    new_skull_mask = cv2.morphologyEx(new_skull_mask, cv2.MORPH_DILATE, kernel,iterations = 1)
    ### now the skull obtained from supervised approach had no gaps, the gaps are actually part of skin
    skin_gaps = skull_mask - new_skull_mask*255
    ### adding the skin gaps to the detected skin from approach 2
    skin_gaps = np.where(skin_gaps>0, 1,0)
    new_skin_mask = skin_mask | skin_gaps
    ### adding labels to the out image
    new_image = np.where(air_mask!=0,0,new_image)
    new_image = np.where(new_skin_mask!=0, 1, new_image)
    new_image = np.where(new_skull_mask!=0, 2, new_image)
    new_image = np.where(out_image==3, 3, new_image)
    new_image = np.where(((~inside_skin!=0)&(new_image==0)), 4, out_image)
    new_image = np.where(combined_white_matter!=0, 5, new_image)
    return new_image

In [40]:
### Scoring for all 10 images
predicts_class_by_class = []
predicts_kmeans = []
predicts_combined = []
for i, image in enumerate(images):
    masks = class_wise_segmentation(image)
    kmeans_out = kmeans_segmentation(image)
    combined_out = combined_kmeans_and_class_by_class(image, masks, kmeans_out)
    predicts_class_by_class.append(masks[0])
    predicts_kmeans.append(kmeans_out)
    predicts_combined.append(combined_out)

df_class_by_class = score_images(predicts_class_by_class,labels)
df_kmeans = score_images(predicts_kmeans,labels)
df_combined = score_images(predicts_combined,labels)
df_1 = df_class_by_class.mean(axis=0).reset_index(name = 'score')
df_1['method'] = 'class_by_class'
df_2 = df_kmeans.mean(axis=0).reset_index(name = 'score')
df_2['method'] = 'kmeans'
df_3 = df_combined.mean(axis=0).reset_index(name = 'score')
df_3['method'] = 'combined'
df = pd.concat([df_1,df_2,df_3],axis=0)
df.rename(columns = {'index':'label'},inplace=True)
# print(df)
out = pd.pivot_table(df,values='score',index='method',columns='label')
out

label,0_mean,air,csf,gray matter,skin,skull,white matter
method,,,,,,,
class_by_class,0.894869,0.985423,0.816944,0.906819,0.914524,0.821126,0.924378
combined,0.913848,0.985423,0.816944,0.952760,0.914524,0.821126,0.992313
kmeans,0.536812,0.915233,0.291857,0.819505,0.243762,0.000000,0.950517


In [47]:
## 3D segmentation
### Approach 1: Kmeans 3D
def kmeans3d_segmentation(input_image):
#     input_image = np.asarray(images).reshape((362,434,10))
    h, w, c = input_image.shape
    ### same as 2d, but adding 10 channels
    # reshape to 1D array of 10 channels
    image_3d = input_image.reshape(h*w,c)

    image_3d = np.float32(image_3d)

    ### using cv2 kmeans
    ### In criteria, we first set the algorithm termination criteria, either max no of iterations or desired result 
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 6
    attempts = 10
    ret,label,center = cv2.kmeans(image_3d,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((input_image.shape))
    ### assigning each cluster to a label based on the range of pixels
    out_image = np.zeros_like(result_image)
    out_image = np.where(((result_image>45) & (result_image<55)), 1, out_image )
    out_image = np.where(((result_image>75) & (result_image<95)), 3, out_image )
    out_image = np.where(((result_image>115) & (result_image<135)), 4, out_image )
    out_image = np.where(((result_image>145) & (result_image<160)), 5, out_image )
    out_image = np.where(((result_image>200) & (result_image<210)), 2, out_image )
    
    return out_image

In [48]:
predicts = kmeans3d_segmentation(images)
df = score_images(predicts, labels)
df_kmeans3 = df.mean(axis=0).reset_index()
df_kmeans3.rename(columns={'index':'label',0:'score'},inplace=True)
df_kmeans3

,label,score
0,air,0.551430
1,skin,0.118988
2,skull,0.000000
3,csf,0.144117
4,gray matter,0.426448
5,white matter,0.382965
6,0_mean,0.270658


In [49]:
### Grayscaling to a single image and processing
# Considering the 10 images are in 3d, we are converting the image to a 
# grayscale image by averaging all channels, and processing through the 2d 
# segmentation algorithm. The ground truth will be taken with the pixel_by_pixel mode value. 

In [50]:
### taking mean of the 10 images to get the single channel image 
images = np.asarray(images)
gray_scale = np.mean(images,axis=0).astype(np.uint8)
### taking the mode across the 10 images for each pixel to arrive at a single label image
mode_label = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=labels)
### calling three algorithms
masks = class_wise_segmentation(gray_scale)
kmeans_out = kmeans_segmentation(gray_scale)
combined_out = combined_kmeans_and_class_by_class(gray_scale, masks, kmeans_out)

print('Dice scores using combined kmeans + class_by_class method: \n')
out = score_image(combined_out,mode_label)
print(out)

Dice scores using combined kmeans + class_by_class method: 

        air      skin     skull       csf  gray matter  white matter    0_mean
0  0.986245  0.912427  0.822814  0.771098     0.901995      0.887595  0.880362


In [51]:
out

,air,skin,skull,csf,gray matter,white matter,0_mean
0,0.986245,0.912427,0.822814,0.771098,0.901995,0.887595,0.880362


In [45]:
### Channel by Channel

In [46]:
## third approach is essentially processing each channel through the 2d segemntation and combining the results
images = np.asarray(images)
outs = np.zeros_like(images)
for i,image in enumerate(images):
    masks = class_wise_segmentation(image)
    outs[i,:,:] = masks[0]
### calculating the dice will be essentially calculating the dice of each channel and averaging
df = score_images(outs, labels)
### which gives the same result as 2d segmentation as expected.